# Data Wrangling 1: Data Cleaning

Before starting any sort of data analysis, we must first ensure that the data is in a form that works for our process. This might involve finding a subset of data, changing the shape of the data, or even adding in information that is not immediately present when received. All of this is referred to as **Data Wrangling**. This can also be worded as Data Manipulation. There are three main types of actions that can be taken to reformat the data.
- Data Cleaning
- Data Transformation
- Data Enrichment

When combined, Data Wrangling is going to take up the majority of the Data Analysis workflow. However, it is a critical step in making sure that we perform an analysis that is logically sound and can produce useful insights. However, great care must be taken to ensure that deceptive insights are not created by our manipulation.
> "If you torture the data long enough, it will confess to anything."
—Ronald Coase, winner of a Nobel Prize in Economics

The ethics of Data Science is not a conversation that falls within the scope of this course, but it should be obvious to the reader that we do not want to intentionally deceive anyone that would be consuming out findings. One way to reduce this risk is to be transparent in your analysis and preparation. Providing information on how you reached your conclusions will give those that receive your results the chance to make their own judgements about your findings.

In this lesson, we will the workflows around data cleaning, with the other two being covered in the next lesson.

## Data Cleaning
If you have read through the previous notebooks, you will find a few examples of how we have performed data cleaning. Below is a list of some common steps that might be taken in a data cleaning process.
- Renaming columns
- Sorting and Reordering rows
- Data type conversions
- Deduplicating data
- Addressing missing or invalid data
- Filtering to the desired subset of data

This is by no means and exhaustive list, nor should you expect to perform these steps in this exact order. These steps are simply a set of blocks that you have at your disposal when building the entire pipeline for your data. This notebook strives only to showcase the functionality of Pandas and to teach the syntax for each of these functions. The steps, their order, and how many repetitions of each step will depend entirely on the data set in question, the desired format that for the final deliverable, and intended use case for your data.


In [ ]:
import pandas as pd

### Renaming Columns
Renaming columns could be one of your steps, especially if the original data has columns that are not very readable. For example, you may receive a spreadsheet with columns that make heavy use of acronyms related to the retrieval process. You might choose to rename the column to something that is a bit descriptive, e.g. rename "pow" to "place-of-work" (yes, this is a real example from a real government CSV that I've seen). Let's take an example from the book *Hands-On Data Analysis with Pandas*.

We first read in some temperature data for New York City.

In [ ]:
temp_ny = pd.read_csv("https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas/master/ch_03/data/nyc_temperatures.csv")
temp_ny.head()

This table is in something called a long format. Notice the column `datatype`. This column is a little problematic, as data type has a couple of possible definitions to that could be used. From reading the documentation (or textbook) we know this provides more information about what the `value` column is measuring. Additionally, we know that in the `value` column is a temperature in Celsius. We also know that the `attributes` column is a collection of flags that gives us information about the data collection process. We can rename those two columns to make reading the table more clear.

In [ ]:
temp_ny.rename(
    columns={
        'value': 'temp_C',
        'attributes': 'flags',
        'datatype': 'temp_type'
    },
    inplace=True
)
temp_ny.head()

In the above cell, we passed a dictionary object into the `columns` argument. Using the syntax `{ 'current_name': 'new_name' }`, we can rename as many columns as we need simultaneously. This instantly makes our dataframe much more intuitive and human readable. Additionally, we have used the `inplace=True` argument to signify that we want to rename the columns in the existing dataframe. Without this value, we would have created a new dataframe that needs to be saved to a variable.

We can also pass in functions as a way of transforming all the column names at the same time.

In [ ]:
print("before: \n", temp_ny.columns)
print("after: \n", temp_ny.rename(str.upper,axis='columns').columns)

### Changing the Index
In addition to renaming the columns of a dataframe, we can also change the index. The index is a special column of the dataframe that is used to reference specific rows. For example, by using `loc`, we can reference specific rows.

In [ ]:
temp_ny.loc[5:10]

However, we can also use other columns as our index. For example, we can set the date to be the index of this dataframe.

In [ ]:
date_index = temp_ny.set_index('date')
date_index.head()

With the date as the index, we can also reference the ranges of dates instead of needing to find the integer index for those rows.

In [ ]:
date_index['2018-10-01':'2018-10-10']

In [ ]:
date_index['2018-10-01':'2018-10-02']

One downside is that this index is still a string object, which we will talk about how to convert in the next section.

*Note*: The above syntax `'2018-10-01':'2018-10-02'` when used with an array like object (Python Lists, NumPy Arrays, or Pandas Dataframes are 3 examples of array like objects) is referred to as **slicing**. Note that there are no entries in the returned data frame that contain the index `'2018-10-02'`. Python will include everything from the first index up until the second index. If we were using numeric arrays like `[2:5]`, you would get the values for 2, 3, and 4, but not 5. This differs when using `loc` on a data frame, as we see above with `temp_ny.loc[5:10]`. This way of calling the columns includes everything from index 5 through (and including) index 10. 

### Type Conversion
When pandas reads in a CSV, it attempts to dynamically interpret the data types of each column. Below is the data types that Pandas interpreted from the CSV we read in.

In [ ]:
temp_ny.dtypes

Note that all columns were assigned the data type object (or string) except for the temperature. If `temp_c` (originally `value`) contained both numeric and string values, it certainly would have been interpreted as object as well. 

Another observation is that our date column is being interpreted as a string. In most cases, we will want this as the datatime object built into pandas which can offer us a lot more functionality when it comes to sorting, ordering, and aggregating. Let's convert this column by using the `to_datetime` function in the Pandas library.

In [ ]:
temp_ny_copy = temp_ny.copy()
temp_ny_copy['date'] = pd.to_datetime(temp_ny_copy.date)
temp_ny_copy.dtypes

In [ ]:
temp_ny_copy.head()

Now our `date` column is a more accurate datatime, which allows us to find some perform more useful functions such as

In [ ]:
temp_ny_copy.date.describe(datetime_is_numeric=True) # datetime_is_numieric=True is not required, but showcases behavior of future versions of Pandas. 

An issue specific to the `datetime` data type is the existence of time zones. This could be critical if you are measuring data from multiple regions in the world, or if you have servers operating on different timezone standards. Pandas dataframes can manage and convert between time zones if we tell pandas what that timezone is. The one caveat is that this has to be done as the index of the row. Combining our knowledge from the previous section, we can
1. convert a column's data type to `datetime`
2. set that column as the index of the dataframe
3. define the timezone for that index

Below uses the copy of the dataframe we created earlier, where the `date` column's data type has already been converted.

In [ ]:
# temp_ny_copy.set_index('date', inplace=True)
# temp_ny_copy = temp_ny_copy.tz_localize('EST')
temp_ny_copy.head()

Notice the `-05:00` at the end of each index. Eastern timezone is also defined as -5 hours in some contexts because it is 5 hours behind UTC timezone with is defined as +0 hours.

Taking this one step further, we can also convert from one timezone to another. If we needed all of our data converted to UTC for example, we can call `tz_convert` on our dataframe and pass in the timezone code.

In [ ]:
temp_ny_copy = temp_ny_copy.tz_convert('UTC')
temp_ny_copy.head()

Notice now that the tail end of the index has `+00:00` and the timestamp of the index has been changed to `05:00:00`. Also note that much of this can be done from the dataframe initialization as well. In the following example, we interpret the date column as a timestamp, set the index to be that column, set the timezone, and convert it to UTC all in the same line.

In [ ]:
initialize_with_date = pd.read_csv(
    "https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas/master/ch_03/data/nyc_temperatures.csv",
    index_col='date',
    parse_dates=True
).tz_localize('EST').tz_convert('UTC')

In [ ]:
initialize_with_date.head()

The 2 main differences here is the use of the two arguments `index_col` and `parse_dates` and that we called both `tz_localize` and `tz_convert` sequentially.

An alternate method of changing the data types is by the use of `assign`. This method can perform column renaming, data type conversion, new column creation, and even complex operations as column values. We additionally can use the `astype` method to convert between data types, i.e. float to integer. All of this can be done in the same function call (thus saving us computation time). Consider the following example:

In [ ]:
temp_ny.dtypes

In [ ]:
temp_ny['temp_C_whole'] = temp_ny.temp_C.astype('int') # cast floating values to integers


In [ ]:
new_df = temp_ny.assign(
    date=pd.to_datetime(temp_ny.date), # convert to datetime datatype
    temp_C_whole=temp_ny.temp_C.astype('int'), # cast floating values to integers
    temp_F=(temp_ny.temp_C * 9/5) + 32, # convert from Celsius to Fahrenheit
    temp_F_whole=lambda x: x.temp_F.astype('int') # cast the newly created Fahrenheit column to integers
    )
new_df.head()

In [ ]:
new_df.dtypes

We could naturally combine the above `assign` call with the methods to assign the `date` column as the index, set the timezone, etc.

One additional data type I want to call out os the `category` datatype. Using `assign` once more:

In [ ]:
new_df = new_df.assign(
    station=temp_ny.station.astype('category'),
    temp_type=temp_ny.temp_type.astype('category')
)
new_df.head()

In [ ]:
new_df.dtypes

Using this data type allows us to perform data type specific functionality. For example, using `describe` on categorical data gives us the count, number of unique values, the mode, and the number of times it occurs in the dataset.

In [ ]:
new_df.describe(include='category')

### Reordering and Sorting
Depending on your context, it may be useful to order the rows in a specific way. The simplest way to do this is by using `sort_values`. This passes in the column name(s), the order of the sort (ascending or descending), and a few other keyword arguments that can be searched if needed.

In [ ]:
new_df.sort_values(by='temp_C', ascending=True).head(10)

Passing in multiple columns in a list will perform the sort in the order of the columns given.

In [ ]:
new_df.sort_values(by=['temp_C_whole', 'date'], ascending=False).head(10)

We can also sort an index (either the row index or the column index). In the following example, we sort the column names by using the `sort_index` method.

In [ ]:
new_df.sort_index(axis='columns').head()

Above when sorting the rows by value, notice that the index showed the original index of the row. Whenever reordering, filtering, or manipulating rows of data, pandas keeps the original index, i.e., the row at index 5 will be the same data as the 6th line of the original csv (remember that python uses a 0 index). This could leave you with rows that are "out of order" or have missing values in the index. We can simply create a new index at the current value by using `reset_index()`

In [ ]:
new_df.loc[(new_df.temp_C >= 20)
           & (new_df.temp_C <= 25)
           & (new_df.temp_type == 'TAVG')]\
    .sort_values('temp_C')\
    .reset_index()

This pushes the original index to a new column, and create a new numeric index based on the current order and filter in place.

## Closing Thoughts

There are many other methods that could be useful in a data cleaning process. These are only a small handful of things that could be done. Once again, the specifics of your data cleaning proces are going to depend entirely on your data set, how you have retrieved it, knowledge of the measurement methods and tools, knowledge of error on those measurements, and many other pieces of information. Always keep in mind the end questions like the following:
- which data are going to be useful moving forward while still providing a clear picture?
- are there any clear outliers that might be caused by bad measurements or intentionally deceptive data?
- is the data currently in the format that I need it, such as correct units and data types?

Questions like these will guide which processes you undertake.